In [12]:

# =============================================================================
# CONSUMPTION 19 MALAWI VILLAGE
# =============================================================================
import numpy as np
import pandas as pd
import os 
os.chdir('C:/Users/rodri/Dropbox/Chied_Field_June_19/Data/Consumption')
percentiles = [0.05, 0.1, .25, .5, .75, 0.8, 0.9, 0.95, 0.99]
euro_MWK = 806.64

# =============================================================================
# Import data: Data from the field and conversion rates (ISA-LSMS price conversions)
# =============================================================================

data = pd.read_stata("[2] Consumption+Non-Food Consumption_(9-16)_050819.dta", convert_categoricals=False)

##### Create conversion kg matrix(unitxitems) with the exact same names and units labels as in the data:


#item labels data
list_items = ['maizemgaiwa', 'maizerefined', 'maizemadeya', 'maizegrain', 'greenmaize', 'rice', 'cassavatubers', 'wsweetpotatoes', 'osweetpotatoes'
, 'ipotatoes', 'potatocrisps', 'bbean', 'pigeonpea', 'groundnut', 'groundnutf', 'onion', 'cabbage', 'tanaposi', 'leafyvegetables', 'tomato', 'eggs', 
'driedfish', 'fleshfish', 'goat', 'chicken', 'otherpoultry', 'smockedfish', 'mango', 'banana', 'guava', 
'wildfruits',  'sugar', 'sugarcane', 'cookingoil', 'softdrinks',
'thobwa', 'locallybrewed', 'salt']

# As it stands today (7/16/19) no one consumed otherpoultry, mango. That means no variables with such label. Need to drop:
list_items.remove('otherpoultry')

list_items.remove('mango')



'''
# NOW I ALREADY CREATED IT THE CONVERSION FILE SO NO NEED TO RUN IT AGAIN ===============
data_conversion = pd.read_excel('conversionkgrates_consumption_malawi_short.xls')

data_conversion = pd.read_stata('ConversionFactorsphysical.dta')
data_conversion = data_conversion.loc[data_conversion.region=='South',['hh_g02', 'hh_g03b','ihps_plus']]

items_conv = data_conversion.item_label.unique()
# Item labels conversion file
list_items_conv = ['Maize Mgaiwa', 'Maize refined', 'Maize Madeya', 'Maize grain',
       'Greenmaize', 'Rice', 'Cassava tubers',
       'White sweet potatoes', 'Orange sweet potatoes', 'Irish Potatoe',
       'Potatoe Chips', 'Bean, brown', 'Pigeon peas (ndolo)', 'Groundnut',
       'Groundnut flour', 'Onion', 'Cabbage', 'Tanaposi/Rape',
       'Other cultivated green leafy vegetables', 'Tomatoe', 'Eggs',
       'Dried Fish', 'Fresh Fish', 'Goat', 'Chicken',
       'Other poultry-guinea fowl, doves, etc', 'Smoked Fish', 'Mango',
       'Banana', 'Guava', 'Wild fruit (Masau, Malambe, etc)', 'Sugar',
       'Sugar Cane', 'Cooking oil',
       'Soft drinks(coca-cola, fanta, sprite etc)', 'Thobwa',
       'Locally brewed liquor(kachasu)', 'Salt']




conversionkg = data_conversion[['unit','item_label','conversion_kgs_country']]


# Reshape as: rows:units, columns:crops
conversionkg = conversionkg.replace(list_items_conv, list_items)
conversionkg_pivot = conversionkg.pivot_table(values='conversion_kgs_country',
                                index='unit',
                                columns='item_label')
conversionkg_pivot.loc[99,:] = np.nan
conversionkg_pivot.loc[100,:] = np.nan
conversionkg_pivot.to_csv('conversionkg_isaprices_matrix.csv')
'''

conversionkg_pivot = pd.read_csv('conversionkg_isaprices_matrix_2.csv',  index_col=0)
# =============================================================================
# Generate empty variables
# =============================================================================

#Obtain the names of the variables per each question of item. Question c is monetary question so not conversion to kgs needed
a_var = []
b_var = []
c_var = []
d_var = []

#Generate variable lables in a list
for item in list_items :
    a = item+'_a'
    b= item+'_b'
    c = item+'_c'  ## expenditure 
    d = item+'_d'
   
    a_var.append(a)
    b_var.append(b)
    c_var.append(c)
    d_var.append(d)
    
list_questions = ['a','b','d']

 
# convert all empty observations to 0. I do that to convert empty units to 99. If not, main conversion loop doesn't work.
# Note that empty doesn't necessary mean 0, so we careful at looking the data       
data = data.stack().apply(pd.to_numeric, errors='ignore').fillna(0).unstack()



In [13]:

# Drop nan observations. Also drop unit 25 (number not in our choices). Also drop 24(whole chicken). Drop empty cells observations
print('UNITS SELECTED PER ITEM')

for var in list_items:
    for i in range(1,4):   #Loop over unit questions.
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(np.nan, 99)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(25, 99)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(24, 99)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(0, 99)
        print('Reported Units of '+var)
        print(pd.value_counts(data[var+'_unit1']))


UNITS SELECTED PER ITEM
Reported Units of maizemgaiwa
99       68
1        66
17       33
15       21
4        19
7        18
5        11
6         7
21        4
16        2
13        2
other     1
Name: maizemgaiwa_unit1, dtype: int64
Reported Units of maizemgaiwa
99       68
1        66
17       33
15       21
4        19
7        18
5        11
6         7
21        4
16        2
13        2
other     1
Name: maizemgaiwa_unit1, dtype: int64
Reported Units of maizemgaiwa
99       68
1        66
17       33
15       21
4        19
7        18
5        11
6         7
21        4
16        2
13        2
other     1
Name: maizemgaiwa_unit1, dtype: int64
Reported Units of maizerefined
99    88
1     58
7     25
4     23
17    16
15    16
6      8
5      8
13     4
21     3
16     2
2      1
Name: maizerefined_unit1, dtype: int64
Reported Units of maizerefined
99    88
1     58
7     25
4     23
17    16
15    16
6      8
5      8
13     4
21     3
16     2
2      1
Name: maizerefined_unit

In [14]:
#Find the households-questions that reported other units.
df_other_units = pd.DataFrame(columns=['householdid','question', 'other_unit']) 
for var in list_items:
    for i in range(1,4):   #Loop over unit questions.       
        # Find who said other units
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace('other', 100)
        other_units_guy = data.loc[data[var+'_unit'+str(i)]==100, ['householdid', var+'_unit'+str(i), var+'_unit'+str(i)+'_other']]            
        if other_units_guy.empty:
            continue
        else:
            d = {'householdid': other_units_guy.iloc[:,0], 'question': other_units_guy.columns[1], 'other_unit':other_units_guy.iloc[:,2]}   
            row = pd.DataFrame(data=d)
            df_other_units = df_other_units.append(row)
        
#df_other_units.to_csv('other_units_list.csv')
print('All households-item-question combinations that reported "other" units')         
print(df_other_units)

All households-item-question combinations that reported "other" units
    householdid              question other_unit
58           68     maizemgaiwa_unit1        5kg
58           68     maizemgaiwa_unit2        5kg
163         189      maizegrain_unit1    Handful
203         236      maizegrain_unit1    Handful
197         230  wsweetpotatoes_unit1       Pots
..          ...                   ...        ...
74           85   locallybrewed_unit2       25ml
16           21            salt_unit1     Packet
166         192            salt_unit1    Handful
16           21            salt_unit2     Packet
166         192            salt_unit2    Handful

[75 rows x 3 columns]


In [16]:
#%% =============================================================================
#   Convert to kgs:  
# =============================================================================

# Generate kg variables empty
for item in list_items:
    for q in list_questions:
        data[item+'_'+q+'kg']= np.nan

print('a: Total Consumption')
for var in a_var:
    for i in range(len(data)): 
        data.iloc[i,data.columns.get_loc(var+'kg')] = data.iloc[i,data.columns.get_loc(var)]*conversionkg_pivot.loc[int(data.iloc[i,  data.columns.get_loc(item+'_unit1')]), item]
    print(data[[var+'kg']].describe())

print('b: Bought')
for var in b_var:
    for i in range(len(data)): 
        data.iloc[i,data.columns.get_loc(var+'kg')] = data.iloc[i,data.columns.get_loc(var)]*conversionkg_pivot.loc[int(data.iloc[i,  data.columns.get_loc(item+'_unit2')]), item]

print('d: Own-produced')
for var in d_var:
    for i in range(len(data)): 
        data.iloc[i,data.columns.get_loc(var+'kg')] = data.iloc[i,data.columns.get_loc(var)]*conversionkg_pivot.loc[int(data.iloc[i,  data.columns.get_loc(item+'_unit3')]), item]



a: Total Consumption
       maizemgaiwa_akg
count       182.000000
mean          1.852498
std           3.094071
min           0.002000
25%           0.300000
50%           0.760000
75%           2.400000
max          20.000000
       maizerefined_akg
count        163.000000
mean           1.657018
std            2.343754
min            0.001000
25%            0.300000
50%            0.640000
75%            2.100000
max           20.000000
       maizemadeya_akg
count       106.000000
mean          0.609344
std           0.807401
min           0.002500
25%           0.300000
50%           0.300000
75%           0.600000
max           6.000000
       maizegrain_akg
count      109.000000
mean         0.600584
std          2.028976
min          0.001000
25%          0.300000
50%          0.300000
75%          0.300000
max         21.300000
       greenmaize_akg
count       27.000000
mean         1.681111
std          2.213585
min          0.010000
25%          0.450000
50%          0.8000

In [17]:
#%%

#Check 
print('To check if the conversion to kgs works correctly: check for example for beans')           
print(data[['bbean_a', 'bbean_unit1', 'bbean_akg','bbean_b', 'bbean_unit2',  'bbean_bkg', 'bbean_d', 'bbean_unit3', 'bbean_dkg']])


#check total consumption in kgs
data['total_foodkg'] = 0
data['purchased_kg'] = 0
data['ownproduced_kg'] = 0

for item in list_items:
    data['total_foodkg'] += data[item+'_akg'].replace(np.nan, 0)
    data['purchased_kg'] += data[item+'_bkg'].replace(np.nan, 0)
    data['ownproduced_kg'] += data[item+'_dkg'].replace(np.nan, 0)
    
    
sumtotalfoodkg = data[['total_foodkg', 'purchased_kg','ownproduced_kg']].describe()   

print('==== Summary Food Consumption last 7 days in kgs aggregated across items =======')
print('')
print(sumtotalfoodkg)


To check if the conversion to kgs works correctly: check for example for beans
    bbean_a  bbean_unit1  bbean_akg bbean_b  bbean_unit2  bbean_bkg bbean_d  \
0       NaN           99        NaN     NaN           99        NaN     NaN   
1       NaN           99        NaN     NaN           99        NaN     NaN   
2       NaN           99        NaN     NaN           99        NaN     NaN   
3       NaN           99        NaN     NaN           99        NaN     NaN   
4       NaN           99        NaN     NaN           99        NaN     NaN   
..      ...          ...        ...     ...          ...        ...     ...   
247     NaN           99        NaN     NaN           99        NaN     NaN   
248       1           16       0.16       1           16       0.16       0   
249     NaN           99        NaN     NaN           99        NaN     NaN   
250     NaN           99        NaN     NaN           99        NaN     NaN   
251     NaN           99        NaN     NaN         

In [18]:
#%%  CONVERT TO MONETARY VALUE

# Generate price variables
for item in list_items:
        data[item+'_price']= np.nan

# price per household
for item in list_items:
    data[item+'_price'] = data[item+'_c'] / data[item+'_bkg'].replace(0,np.nan)

for item in list_items:
    print('Median Price 1 kg of '+item)
    data['med_price_'+item] = data[item+'_price'].median()
    print(data['med_price_'+item].mean())

## compute consumption in MWK
    
for item in list_items:
    for q in list_questions:
        data[item+'_'+q+'MWK']= np.nan

print('a: Total Consumption')
for item in list_items:
    print(item)
    data[item+'_aMWK'] = data[item+'_akg']*data['med_price_'+item]
    print('Food Consumption in MWK during last 7 days item: '+item)
    print(data[item+'_aMWK'].describe(percentiles=percentiles))
            
print('b: Bought')
for item in list_items:
    print(item)
    data[item+'_bMWK'] = data[item+'_bkg']*data['med_price_'+item]


print('d: Own-produced')
for item in list_items:
    print(item)
    data[item+'_dMWK'] = data[item+'_dkg']*data['med_price_'+item]
    

Median Price 1 kg of maizemgaiwa
1250.000031250004
Median Price 1 kg of maizerefined
2500.0000520833314
Median Price 1 kg of maizemadeya
833.3333333333358
Median Price 1 kg of maizegrain
603.3333246933347
Median Price 1 kg of greenmaize
208.3333376736117
Median Price 1 kg of rice
1000.0
Median Price 1 kg of cassavatubers
166.66666666666652
Median Price 1 kg of wsweetpotatoes
333.33333333333303
Median Price 1 kg of osweetpotatoes
416.6666753472234
Median Price 1 kg of ipotatoes
458.33333767361177
Median Price 1 kg of potatocrisps
1208.3333463541685
Median Price 1 kg of bbean
1111.1111111111175
Median Price 1 kg of pigeonpea
549.9999904000005
Median Price 1 kg of groundnut
666.6666666666661
Median Price 1 kg of groundnutf
458.33333767361177
Median Price 1 kg of onion
166.66666666666652
Median Price 1 kg of cabbage
666.6666666666661
Median Price 1 kg of tanaposi
166.66666666666652
Median Price 1 kg of leafyvegetables
166.66666666666652
Median Price 1 kg of tomato
166.66666666666652
Median

In [19]:
    
#check total consumption
data['c_food'] = 0
data['c_food_purch'] = 0
data['c_food_ownprod'] = 0

for item in list_items:
    data['c_food'] += data[item+'_aMWK'].replace(np.nan, 0)
    data['c_food_purch'] += data[item+'_bMWK'].replace(np.nan, 0)
    data['c_food_ownprod'] += data[item+'_dMWK'].replace(np.nan, 0)

data[['c_food', 'c_food_purch' , 'c_food_ownprod']] = data[['c_food', 'c_food_purch' , 'c_food_ownprod']].replace(0.0,np.nan)


sumcfood= ((data[['c_food', 'c_food_purch' , 'c_food_ownprod']]*4/euro_MWK).replace(0.0,np.nan)).describe(percentiles)  
print('==== Summary Food Consumption at Month level in Euros =======')
print(sumcfood) 


==== Summary Food Consumption at Month level in Euros =======
            c_food  c_food_purch  c_food_ownprod
count   249.000000    243.000000        1.000000
mean    110.644113    136.025487       18.575821
std     255.926043    973.064841             NaN
min       0.088081      0.017563       18.575821
5%        8.594488      3.310027       18.575821
10%      11.992165      5.553903       18.575821
25%      22.497713      9.855698       18.575821
50%      47.461073     19.817735       18.575821
75%      79.618335     43.313416       18.575821
80%      99.334277     53.539787       18.575821
90%     232.605371    177.694581       18.575821
95%     431.305168    377.985223       18.575821
99%    1076.299056   1204.192288       18.575821
max    2430.259408  14900.897860       18.575821


In [20]:
#%%   
# Export dataset
data['hhid'] = data['householdid']

## To rain season
data[['c_food','c_food_purch','c_food_ownprod','total_foodkg', 'purchased_kg','ownproduced_kg']]= data[['c_food','c_food_purch','c_food_ownprod','total_foodkg', 'purchased_kg','ownproduced_kg']]*(4*7)

datacon_short = data[['hhid','c_food','c_food_purch','c_food_ownprod','total_foodkg', 'purchased_kg','ownproduced_kg']]

datacon_short.to_csv('cons_short_19_cvgtokg2.csv', index=False)